## THE CIRCUIT CONFIGURATION

In [2]:
import subprocess
import sys
#subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
#subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])


import json
import ezkl
import onnx
!pip list | grep ezkl
!pip list | grep solc-select
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import psutil
import os
import sys
import time
from datetime import datetime


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

ezkl                      11.4.1
solc-select               1.0.4


### GetScore circuit calibration

In [2]:
model_path = os.path.join('outputs/cam/aggr/modelGetscore460.onnx')
settings_path = os.path.join('outputs/cam/aggr/settings.json')
cal_path = os.path.join('outputs/cam/aggr/calib_dataGetscore460.json')

#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

os.environ["RUST_BACKTRACE"] = "1"

!RUST_LOG=trace
# TODO: Dictionary outputs
start_time = time.time()
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
end_time = time.time()
assert res == True
print("gen_settings done in: ", end_time - start_time, "seconds")

os.environ["RUST_BACKTRACE"] = "1"

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure.
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
start_time = time.time()
res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", max_logrows=16, scales=[2])
#res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources" , max_logrows=20, lookup_safety_margin=2, scales=[1], scale_rebase_multiplier=[10])
end_time = time.time()
print("calibrate_settings done in", end_time - start_time, "seconds")

INFO ezkl.execute 2024-07-05 17:11:49,782 execute.rs:1037 num calibration batches: 20
INFO tract_linalg.arm64 2024-07-05 17:11:49,825 arm64.rs:213 CPU optimisation: AppleM
INFO tract_linalg.arm64 2024-07-05 17:11:49,827 arm64.rs:278 ARMv8.2 mmm_f16 and mmv_f16 activated
INFO tract_linalg.arm64 2024-07-05 17:11:49,834 arm64.rs:298 ARMv8.2 tanh_f16 and sigmoid_f16 activated
INFO tract_linalg.arm64 2024-07-05 17:11:49,838 arm64.rs:310 AMX optimisation activated


gen_settings done in:  0.7239141464233398 seconds


WARNING ezkl.execute 2024-07-05 17:14:18,325 execute.rs:1356 

 <------------- Numerical Fidelity Report (input_scale: 2, param_scale: 2, scale_input_multiplier: 10) ------------->

+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+-----------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 2.3422804  | 2.1055791    | 3.2862074 | 1.339046  | 2.3422804      | 2.1055791        | 3.2862074     | 1.339046      | 5.743559           | -1.7019446         | 2.1548018              |
+------------+--------------+-----------+-----------+---------

calibrate_settings done in 148.69529104232788 seconds


In [3]:
# Visualize the circuit settings
json_file_path = settings_path
# Read the content of the JSON file
with open(json_file_path, 'r') as file:
    json_content = json.load(file)

# Print the JSON content
print(json.dumps(json_content, indent=2))

{
  "run_args": {
    "tolerance": {
      "val": 0.0,
      "scale": 1.0
    },
    "input_scale": 2,
    "param_scale": 2,
    "scale_rebase_multiplier": 10,
    "lookup_range": [
      -42,
      64
    ],
    "logrows": 16,
    "num_inner_cols": 2,
    "variables": [
      [
        "batch_size",
        1
      ]
    ],
    "input_visibility": "Private",
    "output_visibility": "Public",
    "param_visibility": "Fixed",
    "div_rebasing": false,
    "rebase_frac_zero_constants": false,
    "check_mode": "UNSAFE",
    "commitment": "KZG"
  },
  "num_rows": 106737,
  "total_assignments": 213474,
  "total_const_size": 4324,
  "total_dynamic_col_size": 0,
  "num_dynamic_lookups": 0,
  "num_shuffles": 4320,
  "total_shuffle_col_size": 17280,
  "model_instance_shapes": [
    [
      1,
      1
    ]
  ],
  "model_output_scales": [
    6
  ],
  "model_input_scales": [
    2,
    2
  ],
  "module_sizes": {
    "polycommit": [],
    "poseidon": [
      0,
      [
        0
      ]
    ]


### GetScore Setup

In [ ]:
# Compile the model into a circuit
compiled_model_path = os.path.join('outputs/cam/aggr/getscore460.ezkl')
#compiled_model_path = os.path.join('prover/outputs/cam/aggr/getscore.ezkl')

res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

# Get public srs from kzg ceremony.
settings_path = os.path.join('outputs/cam/aggr/settings.json')
res = ezkl.get_srs(settings_path)

INFO ezkl.execute 2024-07-05 17:14:42,451 execute.rs:741 SRS already exists at that path


INFO ezkl.execute 2024-07-05 17:14:42,455 execute.rs:639 read 8388868 bytes from file (vector of len = 8388868)
INFO ezkl.execute 2024-07-05 17:14:42,487 execute.rs:646 file hash: 2a1a494630e71bc026dd5c0eab4c1b9a5dbc656228c1f0d48f5dbd3909b161d3


In [5]:
# Setup the circuit and make sure the keys are generated afterwards.
import os
import time
import psutil

pk_path = os.path.join('outputs/cam/aggr/pk(getscore)460.key')
vk_path = os.path.join('outputs/cam/aggr/vk(getscore)460.key')

# Start time and process
start_time = time.time()
process = psutil.Process(os.getpid())

os.environ["RUST_BACKTRACE"] = "1"

# Capture initial CPU and RAM usage
initial_cpu_times = process.cpu_times()
initial_memory_info = process.memory_info()

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

# Capture end CPU and RAM usage
final_cpu_times = process.cpu_times()
final_memory_info = process.memory_info()

end_time = time.time()
execution_time = end_time - start_time

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

# Calculate CPU and RAM usage
cpu_user_time = final_cpu_times.user - initial_cpu_times.user
cpu_system_time = final_cpu_times.system - initial_cpu_times.system
ram_usage_mb = (final_memory_info.rss - initial_memory_info.rss) / (1024 * 1024)  

print("setup done in: ", execution_time, "seconds")
print("CPU user time: ", cpu_user_time, "seconds")
print("CPU system time: ", cpu_system_time, "seconds")
print("RAM usage: ", ram_usage_mb, "MB")

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO ezkl.graph.model 2024-07-05 17:15:19,664 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-05 17:15:24,183 mod.rs:520 VK took 4.526
INFO ezkl.graph.model 2024-07-05 17:15:24,190 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-05 17:15:28,412 mod.rs:526 PK took 4.228
INFO ezkl.pfsys 2024-07-05 17:15:28,415 mod.rs:844 done saving verification key ✅
INFO ezkl.pfsys 2024-07-05 17:15:28,815 mod.rs:827 done saving proving key ✅


setup done in:  9.19367504119873 seconds
CPU user time:  29.109272575999995 seconds
CPU system time:  14.336196608000023 seconds
RAM usage:  603.984375 MB


## CAM Proving
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [6]:
witnesses_dir = os.path.join('outputs/cam/aggr/witnesses/460')
inputs_dir = os.path.join('outputs/cam/aggr/inputs/460')
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def generate_witness(input_file):
    data_path = os.path.join(inputs_dir, input_file)
    witness_file = f'witness_{os.path.splitext(input_file)[0]}.json'
    witness_path = os.path.join(witnesses_dir, witness_file)
    
    try:
        res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
        print(f'Generated witness for {input_file}: {res}')
    except Exception as e:
        print(f'Failed to generate witness for {input_file}: {e}')
        
input_files = [f for f in os.listdir(inputs_dir) if f.endswith('.json')]

# Generate a witness for each input file
async def main():
    tasks = [generate_witness(input_file) for input_file in input_files]
    await asyncio.gather(*tasks)

# Run the main function within the existing event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
        

Generated witness for input460_1.json: {'inputs': [['...']], 'outputs': [['da01000000000000000000000000000000000000000000000000000000000000']], 'max_lookup_inputs': 25, 'min_lookup_inputs': -31, 'max_range_size': 0}
Generated witness for input460_0.json: {'inputs': [['...']], 'outputs': [['7c02000000000000000000000000000000000000000000000000000000000000']], 'max_lookup_inputs': 29, 'min_lookup_inputs': -35, 'max_range_size': 0}
Generated witness for input460_2.json: {'inputs': [['...']], 'outputs': [['b700000000000000000000000000000000000000000000000000000000000000']], 'max_lookup_inputs': 29, 'min_lookup_inputs': -26, 'max_range_size': 0}


In [7]:
# Generate the proofs
proofs_dir = 'outputs/cam/aggr/proofs/460'  
os.makedirs(proofs_dir, exist_ok=True)
all_proofs = []
# Directory to save proofs
# Function to generate proof for a single witness file
def generate_proof(witness_file):
    witness_path = os.path.join(witnesses_dir, witness_file)
    proof_file = f'proof_{os.path.splitext(witness_file)[0]}.json'
    proof_path = os.path.join(proofs_dir, proof_file)

    start_time = time.time()
    process = psutil.Process(os.getpid())  
    start_memory = process.memory_info().rss 
    start_cpu_times = process.cpu_times()  

    try:
        proof = ezkl.prove(
            witness_path,
            compiled_model_path,
            pk_path,
            proof_path,
            "for-aggr"  # To produce an aggregated EVM proof using poseidon for the smaller proofs
        )
        end_time = time.time()
        execution_time = end_time - start_time
        end_memory = process.memory_info().rss 
        memory_usage = (end_memory - start_memory) / (1024 * 1024)  
        end_cpu_times = process.cpu_times() 

        # Calculate CPU times
        cpu_user_time = end_cpu_times.user - start_cpu_times.user
        cpu_system_time = end_cpu_times.system - start_cpu_times.system

        print(f'Proof generated for {witness_file} in {execution_time} seconds')
        print(f'Memory used for {witness_file}: {memory_usage:.2f} MB')
        print(f'CPU user time for {witness_file}: {cpu_user_time} seconds')
        print(f'CPU system time for {witness_file}: {cpu_system_time} seconds')
        print(proof)
        
        all_proofs.append(proof_path)
        return execution_time, memory_usage, cpu_user_time, cpu_system_time
    except Exception as e:
        print(f'Failed to generate proof for {witness_file}: {e}')

# Get a list of all witness JSON files in the witnesses directory
witness_files = [f for f in os.listdir(witnesses_dir) if f.endswith('.json')]

# Generate a proof for each witness file
cumulative_time = 0
cumulative_memory = 0
cumulative_cpu_user_time = 0
cumulative_cpu_system_time = 0

for witness_file in witness_files:
    execution_time, memory_usage, cpu_user_time, cpu_system_time = generate_proof(witness_file)
    cumulative_time += execution_time
    cumulative_memory += memory_usage
    cumulative_cpu_user_time += cpu_user_time
    cumulative_cpu_system_time += cpu_system_time

print(f'Cumulative time for generating all proofs: {cumulative_time:.2f} seconds')
print(f'Cumulative memory used for generating all proofs: {cumulative_memory:.2f} MB')
print(f'Cumulative CPU user time for generating all proofs: {cumulative_cpu_user_time:.2f} seconds')
print(f'Cumulative CPU system time for generating all proofs: {cumulative_cpu_system_time:.2f} seconds')

INFO ezkl.pfsys 2024-07-05 17:15:50,025 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-05 17:15:50,036 mod.rs:581 proof started...
INFO ezkl.graph.model 2024-07-05 17:15:50,049 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-05 17:16:03,067 mod.rs:620 proof took 13.29


Proof generated for witness_input460_1.json in 13.528788089752197 seconds
Memory used for witness_input460_1.json: 213.62 MB
CPU user time for witness_input460_1.json: 47.86092441599996 seconds
CPU system time for witness_input460_1.json: 8.505556991999924 seconds
{'instances': [['da01000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x6265e566d346104a162dbb52afcd6e8c29d768f3aef0da7a6f174afd5a6402102557f33c31575ae26ad0812c658eec3db34ff34460aecaf181ce84d899b3b46422ef52b29c231f5f335918cb68c0b12018dd7da9b2db9660ca5a7b7cd8643930a059b57402d2a337f992d4d23965d538b43c0e8a04392d08e0b57de67e3c3a1647edd618bafaeb66266b2a9c75133492bd17beca494b70298102019d1cde2d441aae70b121154cf27207e4325c1d48b1c313ea720f2c6a3777685b2cc4e4a1085fa5178ca8d3486895a424c9e5e5769cb13c40b5aa42ac831cbeda37d440a611d16af074a39c5667ac55f17b6032e3dd311a312b31e66b02f20f980018357d0b8432bc9e50dfeee76ce530d4ce9c3779634a393a7e4b38d39eeb7c4efa5f91695c1763b334cdca5c9259421f9a474263bd0d8575b30e4e72eaf396d832b01

INFO ezkl.pfsys 2024-07-05 17:16:03,523 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-05 17:16:03,535 mod.rs:581 proof started...
INFO ezkl.graph.model 2024-07-05 17:16:03,550 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-05 17:16:16,415 mod.rs:620 proof took 12.879


Proof generated for witness_input460_0.json in 13.349061250686646 seconds
Memory used for witness_input460_0.json: 67.45 MB
CPU user time for witness_input460_0.json: 47.622422528000016 seconds
CPU system time for witness_input460_0.json: 8.511324159999958 seconds
{'instances': [['7c02000000000000000000000000000000000000000000000000000000000000']], 'proof': '0xfbd91bb51b08431426aa377347fd179feaa0f1418bd1940fe92c80677741fc410f1906d135605191dc178784ff20cdbb9e734d0dccc461a6898ba8bed335b80c7d8bc6d72e535cff7dfe99aa02cbbc36d60bd3b5583cce5c840d81a8dcfa6d17cc8efb785ad096b3267d63c634f52cd115a261d8ff0fbaf18df8abab38929520f3ae7e84894646966c3dcccc3a4e3ac9b66b3f45b2b49e322fb189e0cc8159278eea2b032f55532403f6d8ce6026076b3f3e18fef40dafa73bf209e92602740abcd6fc6376c086261479ca37fb462492d9941125b814982847a1d1804497cd2c5066340f3fecb5cb258a55e834ae95a9ace9170c4055cab6186de2b07d34610601b114be06de18cf5e70407cea4a54c36c46470e9ca56642b372738617ea30651b3d285c66d71165b697723577e66a7e7328ce49b89a903275d1502a91c64

INFO ezkl.pfsys 2024-07-05 17:16:16,902 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-05 17:16:16,913 mod.rs:581 proof started...
INFO ezkl.graph.model 2024-07-05 17:16:16,929 model.rs:1083 model layout...
INFO ezkl.pfsys 2024-07-05 17:16:30,209 mod.rs:620 proof took 13.295


Proof generated for witness_input460_2.json in 13.79973292350769 seconds
Memory used for witness_input460_2.json: 68.27 MB
CPU user time for witness_input460_2.json: 47.79117772799998 seconds
CPU system time for witness_input460_2.json: 8.287944703999983 seconds
{'instances': [['b700000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x3377f48bcbbcc8ebcc0ea3a8a9ee5237b4a5d1c122a9d10f44261c802f714b6d5128c5f554335e6a9a75534d420f2a9ef529420ac52f986e7ac911baec7f516c2ff28a0ed1bd8216ccad3dc2390f091f79bda21e9022296775dc875027d4591899cc035dd9b125b0a66d8be9f111cab055bc2616f2ca94c05939e1afbd3c0944eeeadf062b286107f82c0e88b5546ca77a76177790ae2ab8ec319bca311b7a09b45711a3a13af1e7ff46e02d155fe7a7f5e9f6bb15c774b6239d3fe82ca2ff4072aaee4a474a581d349a34f644a91f5c154df3bf026e4bb9afaa308b6a26d3284524a88a9ded8b39fe782aa5444a4fb3c6ff8ab2d11609e01166006cb2a4ae1352f525348ae55b324f6e877b7887bbed4fded164d8a065d3baa7a9730a91451bcb9d1ecf1c5fc18665224a785acc2e81312c09b8e3ec9bcc46369ef10c4f2a2

In [8]:
# Generate a larger SRS. This is needed for the aggregated proof

res = await ezkl.get_srs(settings_path=None, logrows=24, commitment=ezkl.PyCommitments.KZG)

INFO ezkl.execute 2024-07-05 17:17:11,568 execute.rs:741 SRS already exists at that path
INFO ezkl.execute 2024-07-05 17:17:12,179 execute.rs:639 read 2147483908 bytes from file (vector of len = 2147483908)
INFO ezkl.execute 2024-07-05 17:17:18,328 execute.rs:646 file hash: f94fa4afa2f5147680f907d4dd96a8826206c26bd3328cd379feaed614b234de


In [4]:
import os

def get_json_file_paths(proofs_dir):
    json_file_paths = []
    for root, dirs, files in os.walk(proofs_dir):
        for file in files:
            if file.endswith('.json'):
                json_file_paths.append(os.path.join(root, file))
    return json_file_paths

proofs_dir = 'outputs/cam/aggr/proofs/460'
all_proofs = get_json_file_paths(proofs_dir)
print(all_proofs)

['outputs/cam/aggr/proofs/460/proof_witness_input460_2.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_0.json', 'outputs/cam/aggr/proofs/460/proof_witness_input460_1.json']


In [10]:
# Run mock aggregate to check whether the proof works
# Use mock to check for validity as it takes a shorter time to check compared to a full aggregated proof

res = ezkl.mock_aggregate(all_proofs, 23)
assert res == True

In [11]:
# Setup the vk and pk for aggregate
import os
import time
import psutil

# Setup the vk and pk for aggregate
aggregate_vk_path = os.path.join('outputs/cam/aggr/aggr460.vk')
aggregate_pk_path = os.path.join('outputs/cam/aggr/aggr460.pk')

# Start tracking time and resource usage
start_time = time.time()
process = psutil.Process(os.getpid())
start_cpu_time = process.cpu_times()
start_memory_usage = process.memory_info().rss

res = ezkl.setup_aggregate(
    all_proofs,
    aggregate_vk_path,
    aggregate_pk_path,
    23
)

# End tracking time and resource usage
end_time = time.time()
end_cpu_time = process.cpu_times()
end_memory_usage = process.memory_info().rss

# Calculate elapsed time and resource usage
execution_time = end_time - start_time
cpu_time = (end_cpu_time.user + end_cpu_time.system) - (start_cpu_time.user + start_cpu_time.system)
ram_usage = (end_memory_usage - start_memory_usage) / (1024 * 1024)  # Convert to MB

# Print the results
print(f'Time for setup_aggregate: {execution_time:.2f} seconds')
print(f'CPU time for setup_aggregate: {cpu_time:.2f} seconds')
print(f'RAM usage for setup_aggregate: {ram_usage:.2f} MB')

assert os.path.isfile(aggregate_vk_path)
assert os.path.isfile(aggregate_pk_path)

INFO ezkl.pfsys 2024-07-05 17:26:07,879 mod.rs:520 VK took 91.471
INFO ezkl.pfsys 2024-07-05 17:30:03,202 mod.rs:526 PK took 235.306
INFO ezkl.pfsys 2024-07-05 17:30:03,250 mod.rs:844 done saving verification key ✅
INFO ezkl.pfsys 2024-07-05 17:33:34,051 mod.rs:827 done saving proving key ✅


Time for setup_aggregate: 538.97 seconds
CPU time for setup_aggregate: 1483.69 seconds
RAM usage for setup_aggregate: -1910.36 MB


In [13]:
aggregate_vk_path = os.path.join('outputs/cam/aggr/aggr460.vk')
aggregate_pk_path = os.path.join('outputs/cam/aggr/aggr460.pk')

vk_size = os.path.getsize(aggregate_vk_path) / (1024 * 1024)
pk_size = os.path.getsize(aggregate_pk_path) / (1024 * 1024)

print(f'VK file size: {vk_size:.2f} MB')
print(f'PK file size: {pk_size:.2f} MB')


VK file size: 2.00 MB
PK file size: 33794.00 MB


In [15]:
import os
import psutil
import time

def log_process_stats(process):
    cpu_time = process.cpu_times()
    memory_info = process.memory_info()
    return {
        "user_cpu_time": cpu_time.user,
        "system_cpu_time": cpu_time.system,
        "memory_used": memory_info.rss / (1024 ** 2)  # Convert bytes to MB
    }

# Run aggregate proof
aggregate_proof_path = os.path.join('outputs/cam/aggr/aggr460.pf')

# Start measuring time and resource usage
process = psutil.Process(os.getpid())
start_time = time.time()

res = ezkl.aggregate(
    all_proofs,
    aggregate_proof_path,
    aggregate_pk_path,
    "evm",
    23,
    "safe"
)

# End measuring time and resource usage
end_time = time.time()
stats = log_process_stats(process)
stats["elapsed_time"] = end_time - start_time

assert os.path.isfile(aggregate_proof_path)

# Log size of the aggregated proof
proof_size = os.path.getsize(aggregate_proof_path) / (1024 ** 2)  # Convert bytes to MB

# Print logs
print(f"User CPU time: {stats['user_cpu_time']} seconds")
print(f"System CPU time: {stats['system_cpu_time']} seconds")
print(f"Memory used: {stats['memory_used']} MB")
print(f"Elapsed time: {stats['elapsed_time']} seconds")
print(f"Size of aggregated proof: {proof_size} MB")

INFO ezkl.pfsys 2024-07-05 17:49:39,537 mod.rs:810 loaded proving key ✅
INFO ezkl.pfsys 2024-07-05 17:49:41,248 mod.rs:581 proof started...
INFO ezkl.pfsys 2024-07-05 18:01:32,451 mod.rs:620 proof took 711.183
INFO ezkl.execute 2024-07-05 18:01:33,194 execute.rs:2320 Aggregation proof took 743.111


User CPU time: 6992.259710976 seconds
System CPU time: 959.76357888 seconds
Memory used: 332.5625 MB
Elapsed time: 743.1409840583801 seconds
Size of aggregated proof: 0.01743316650390625 MB


In [8]:
# Check if the proof is valid
import psutil
aggregate_vk_path = os.path.join('outputs/cam/aggr/aggr460.vk')

process = psutil.Process(os.getpid())  # Get the current process
start_time = time.time()
start_cpu_times = process.cpu_times()  # CPU times at the start
start_memory = process.memory_info().rss  # Memory usage at the start

res = ezkl.verify_aggr(
    'outputs/cam/aggr/aggr460.pf',
    aggregate_vk_path,
    23,
)


end_time = time.time()
end_cpu_times = process.cpu_times()  # CPU times at the end
end_memory = process.memory_info().rss  # Memory usage at the end

execution_time = end_time - start_time
cpu_time_user = end_cpu_times.user - start_cpu_times.user
cpu_time_system = end_cpu_times.system - start_cpu_times.system
total_cpu_time = cpu_time_user + cpu_time_system
memory_used = (end_memory - start_memory) / (1024 * 1024)  # Convert to MB

print(f"Proof generation took {execution_time:.2f} seconds (real-time).")
print(f"CPU time used: User - {cpu_time_user:.2f} seconds, System - {cpu_time_system:.2f} seconds.")
print(f"Total CPU time: {total_cpu_time:.2f} seconds.")
print(f"Memory used: {memory_used:.2f} MB")

assert res == True

INFO ezkl.pfsys 2024-07-08 12:02:29,655 mod.rs:787 loaded verification key ✅
INFO ezkl.execute 2024-07-08 12:02:29,659 execute.rs:2453 verify took 0.2
INFO ezkl.execute 2024-07-08 12:02:29,659 execute.rs:2458 verified: true


Proof generation took 0.45 seconds (real-time).
CPU time used: User - 0.25 seconds, System - 0.19 seconds.
Total CPU time: 0.44 seconds.
Memory used: 33.05 MB


In [17]:
# Create a smart contract verifier for the aggregated proof

sol_code_path = os.path.join("outputs/cam/aggr/Verifier.sol")
abi_path = os.path.join("outputs/cam/aggr/Verifier_ABI.json")

res = await ezkl.create_evm_verifier_aggr(
    [settings_path],
    aggregate_vk_path,
    sol_code_path,
    abi_path,
    logrows=23)

INFO ezkl.pfsys 2024-07-05 18:03:14,291 mod.rs:787 loaded verification key ✅


In [12]:
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )

input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


AttributeError: module 'ezkl' has no attribute 'vecu64_to_felt'

In [21]:
onchain_input_array = []

# using a loop
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.felt_to_big_endian(field_element))
        formatted_output += '"' + str(onchain_input_array[-1]) + '"'
        if j != len(value) - 1:
            formatted_output += ", "
    if i != len(proof["instances"]) - 1:
        formatted_output += ", "
formatted_output += "]"

json_data = {
    "instances": formatted_output,
    "proof": proof["proof"]
}

with open('prover/outputs/cam/evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


Saved evmInputs.json:  {'instances': '["..."]', 'proof': '...'}
